# HW3 파이썬프로그래밍 (2023년 2학기) 기말과제
- 분반: ??
- 이름: ???
- 학번: ?????????


----

In [ ]:
def h(n, src, dst, tmp): # n>0, n개 원반 하노이 탑을 src에서 dst로 tmp를 거쳐서 옮기기
    if n > 1:
        return h(n-1,src,tmp,dst) + h(1,src,dst,tmp) + h(n-1,tmp,dst,src)
    else: # n == 1
        return [(src,dst)]

In [ ]:
h(1,'A','C','B')

[('A', 'C')]

In [ ]:
h(3,'A','C','B')

[('A', 'C'),
 ('A', 'B'),
 ('C', 'B'),
 ('A', 'C'),
 ('B', 'A'),
 ('B', 'C'),
 ('A', 'C')]

## 1. (10점) 하노이 탑 함수를 억지로 반복문으로 바꾸기
수업시간에 재귀함수를 *억지로*(즉, 파이썬의 함수 호출을 시뮬레이션해서)
반복문으로 바꾸는 방식으로 위의 재귀함수 `h`에 해당하는 반복문 버전의
`htf`를 작성하라.

코드는 다음과 같은 방식을 따라 작성해야 한다.
(그 외의 창의적인 방법은 이 문제의 정답으로 인정하지 않음.)

앞으로 남은 작업을 기록하는 작업 스택 `ts`에는
다음과 같은 작업이 등록할 수 있다.
- `('h', n, src, dst, tmp)` 재귀함수 호출에 해당하는 작업
  - 파이썬 코드로는 `h(n-1,src,tmp,dst)`에 해당

리턴 값은 작업 스택과 별도의 리턴값 스택 `rs`를 통해 관리한다.
`rs`의 원소는 순서쌍 리스트이다. 그러니까 `rs`는 순서쌍으로 이루어진 리스트들이 여러 개 쌓여있는 셈이다. 예를 들면 다음과 같이 ...
```python
rs = [ [('A','C'),('A','B'),('C','B')],
       [('A','C')],
       [('B','A'),('B','C'),('A','C')] ]
```

In [ ]:
def htf(n, src, dst, tmp):
    ts = [('h', n, src, dst, tmp)]
    rs = []
    roof = n
    arr = []
    while ts != []:
        task = ts.pop() # 마지막에 등록된 작업 뽑아내기
        if 'h' == task[0]:
            # 마지막에 등록된 작업이 h인 경우는 이렇게
            _,n,src,dst,tmp = task
            if n > 1:
                if n == roof:
                    ts.append( ('i') )
                ts.append( ('h',n-1,tmp,dst,src) ) # 1단계
                if n == roof:
                    ts.append( ('i') )
                ts.append( ('p',src,dst) )         # 2단계
                if n == roof:
                    ts.append( ('i') )
                ts.append( ('h',n-1,src,tmp,dst) ) # 3단계

            else: # n == 1
                ts.append( ('p',src,dst) ) # 출력 함수 호출 작업 등록
        elif 'p' == task[0]: # 마지막에 등록된 작업이 p인 경우는
            _,src,dst = task
            arr.append((src, dst))
        else:
            rs.append(arr.copy())
            arr.clear()

    return rs

In [ ]:
def h(n, src, dst, tmp): # n개 원반 하노이 탑을 src에서 dst로 tmp를 거쳐서 옮기기
    if n > 1:
        h(n-1, src, tmp, dst) # 1단계
        print(src, "=>", dst) # 2단계
        h(n-1, tmp, dst, src) # 3단계
    else: # n == 1
        print(src, "=>", dst)

In [ ]:
htf(3, 'A', 'C', 'B')

[[('A', 'C'), ('A', 'B'), ('C', 'B')],
 [('A', 'C')],
 [('B', 'A'), ('B', 'C'), ('A', 'C')]]

----

아래와 같은 공식이 성립하는 무한 다항식의 곱셈과 나눗셈을 정의하자

- (공식1): $1 = (1-x) \times (1 + x + x^2 + x^3 + x^4 + \cdots)$
- (공식2): $1 = (1+x) \times (1 - x + x^2 - x^3 + x^4 + \cdots)$
- (공식3): $1/(1-x) = 1 + x + x^2 + x^3 + x^4 + \cdots$
  - 공식3은 공식1에 대응되는 나눗셈 버전
- (공식4): $1/(1+x) = 1 - x + x^2 - x^3 + x^4 + \cdots$
  - 공식4는 공식2에 대응되는 나눗셈 버전
- (공식5):
$ 1/(1-x^2) = (1 + x + x^2 + x^3 + x^4 + \cdots) \times (1 - x + x^2 - x^3 + x^4 + \cdots)$
  - 공식3과 공식4의 양변을 각각 곱하면 얻을 수 있는 공식

In [ ]:
from itertools import *
# 곱셈과 나눗셈을 작성할 때 활용할 뎃셈과 뺄셈

def add_poly(p1, p2): # 무한 다항식의 덧셈
    return map( (lambda a,b: a+b), p1, p2)

def sub_poly(p1, p2): # 무한 다항식의 뺄셈
    return map( (lambda a,b: a-b), p1, p2)

## 2. (10점) 무한 다항식의 곱셈을 작성하고 테스트하라
제너레이터 함수와 제너레이터 표현식
그리고 관련 라이브러리 함수들을 활용해
작성하라.

테스트할 내용은 다음과 같다.
- 공식1이 성립하는지 확인
- 공식2가 성립하는지 확인
- 제시된 공식들 외에 확인하고 싶은 무한 다항식의 곱셈 추가 테스트 2개 이상

이렇게 총 4개 이상의 테스트를 해야 한다

In [ ]:
from itertools import *

def mul_poly(p1, p2):
    pass

In [ ]:
# 공식1 테스트

In [ ]:
# 공식2 테스트

In [ ]:
# 자유 테스트

In [ ]:
# 자유 테스트

## 3. (10점) 무한 다항식의 나눗셈을 작성하고 테스트하라

제너레이터 함수와 제너레이터 표현식
그리고 관련 라이브러리 함수들을 활용해
작성하라.

테스트할 내용은 다음과 같다.
- 공식3이 성립하는지 확인
- 공식4가 성립하는지 확인
- 공식5가 성립하는지 확인 (앞 문제의 곱셉 함수도 함께 활용)
- 제시된 공식들 외에 확인하고 싶은 무한 다항식의 나눗셈 추가 테스트 1개 이상

이렇게 총 4개 이상의 테스트를 해야 한다

In [ ]:
from itertools import *

def div_poly(p1, p2):
    pass

In [ ]:
# 공식3 테스트

In [ ]:
# 공식4 테스트

In [ ]:
# 공식5 테스트

In [ ]:
# 자유 테스트

---

최장 공통 부분문자열 (Longest Common Substring, 줄여서 LCSstr이라고 하자) 표현하는 재귀 알고리즘과 그에 대해 메모해두기를 적용한 함수를 작성해 보자.


In [ ]:
from typing import *

In [ ]:
s5 = "hello"
s5[1:], s5[:-1]

('ello', 'hell')

In [ ]:
# s1과 s2의 LCSstr 길이를 계산하는 재귀함수
def lcsstr(s1:str, s2:str) -> int:
    if s1 != "" and s2 != "":
        if s1 == s2:
            return len(s1)
        else:
            _s1, s1_ = s1[1:], s1[:-1]
            _s2, s2_ = s2[1:], s2[:-1]
            n1 = lcsstr(_s1,s2)
            n2 = lcsstr(s1_,s2)
            n3 = lcsstr(s1,_s2)
            n4 = lcsstr(s1,s2_)
            # 두 문자열의 양 끝 중 한 곳에서 글자 하나 제외하는 모든 4가지 경우 중 최대값
            m = max(n1, n2, n3, n4)
            return m
    else: # s1=="" or s2==""
        return 0

In [ ]:
lcsstr("", "hello")

0

In [ ]:
lcsstr("hello", "")

0

In [ ]:
lcsstr("hello", "lolly")

2

In [ ]:
lcsstr("hello", "jelly")

3

In [ ]:
lcsstr("hello", "shell")

4

In [ ]:
lcsstr("hello", "hello")

5

In [ ]:
lcsstr("hellohi", "hihello") # 느려진다 (대략 5초 이상)

5

In [ ]:
# lcsstr("hellobye", "byehello") # 너무 느리다 (1분 훌쩍 넘어감)
# # 5가 나올 것이다. 시간이 많이 걸리니까 주석처리해 놓음

## 4. (5점) `lcsstr` 함수에 메모해두기 적용
`lcsstr` 함수에 메모해두기를 적용한 `lcsstrM` 함수를 작성하라.

당연히 `functools`의 `@cache` 매크로를 활용하면 안되고,
수업시간에 했던 것처럼 메모해두는 용도의 딕셔너리와
내부 함수 `rec`을 정의하여 `lcsstr`의 구조를 되도록
따르면서 작성하라.  

함수를 작성한 다음 충분한 개수의 적절한 테스트를 하라.

In [ ]:
def lcsstrM(s1:str, s2:str) -> int:
    # 메모해두기에 활용할 딕셔너리 여기에 초기화
    def rec(s1:str, s2:str) -> int:
        pass # lcsstr의 구조를 따라 작성
    # rec 함수 호출
    return rec(s1, s2)

In [ ]:
# 충분한 개수의 적절한 테스트를 하라.

In [ ]:
# 충분한 개수의 적절한 테스트를 하라.

In [ ]:
# 충분한 개수의 적절한 테스트를 하라.

In [ ]:
# 충분한 개수의 적절한 테스트를 하라.

## 5. (5점) LCSstr 그 자체(즉, 문자열)를 계산하는 재귀함수
주어진 두 문자열에 대한 LCSstr의 길이를 계산하는 대신
LCSstr 자체를 계산하는 함수 `lcsstrS`를 작성하라.
그러니까 `lcsstrS`의 리턴값은 int가 아니라 str이다.
메모해두기는 적용하지 말고 `lcsstr`의
재귀적인 구조를 그대로 따르면서 작성해야 한다.

서로 다른 여러 개의 LCSstr이 있는 경우는
여러 개 중에서 아무거나 하나 리턴하면 된다.
예컨대, `lcsstrS("hello", "lolly")`의 값으로 `"ll"`을 리턴하도록 해되 좋고 `"lo"`를 리턴하도록 해도 좋다.

함수를 작성한 다음 충분한 개수의 적절한 테스트를 하라.



In [ ]:
# s1과 s2의 LCSstr 그 자체(즉, 문자열)를 계산하는 재귀함수
def lcsstrS(s1:str, s2:str) -> str:
    '''
    # 참고로, 아래는 lcsstr 함수 몸체 내용을 복붙해 놓았다
    # (위로 스크롤하면서 참고하느라 귀찮치 않도록)
    if s1 != "" and s2 != "":
        if s1 == s2:
            return len(s1)
        else:
            _s1, s1_ = s1[1:], s1[:-1]
            _s2, s2_ = s2[1:], s2[:-1]
            n1 = lcsstr(_s1,s2)
            n2 = lcsstr(s1_,s2)
            n3 = lcsstr(s1,_s2)
            n4 = lcsstr(s1,s2_)
            # 두 문자열의 양 끝 중 한 곳에서 글자 하나 제외하는 모든 4가지 경우 중 최대값
            m = max(n1, n2, n3, n4)
            return m
    else: # s1=="" or s2==""
        return 0
    '''
    pass

In [ ]:
# 충분한 개수의 적절한 테스트를 하라.

In [ ]:
# 충분한 개수의 적절한 테스트를 하라.

In [ ]:
# 충분한 개수의 적절한 테스트를 하라.

In [ ]:
# 충분한 개수의 적절한 테스트를 하라.

## 6. (5점) 모든 LCSstr의 집합을 계산하는 재귀함수
하나의 LCSstr이 아니라 모든 LCSstr의 집합을 계산하는 함수 `lcsstrSS`를 작성하라.
그러니까 `lcsstrSS`의 str이 아니라 set이다.
메모해두기는 적용하지 말고 `lcsstrS`의
재귀적인 구조를 그대로 따르면서 작성해야 한다.

예컨대, `lcsstrSS("hello", "lolly") == {"ll","lo"}`이다.

함수를 작성한 다음 충분한 개수의 적절한 테스트를 하라.



In [ ]:
def lcsstrSS(s1:str, s2:str) -> set[str]:
    pass

In [ ]:
# 충분한 개수의 적절한 테스트를 하라.

In [ ]:
# 충분한 개수의 적절한 테스트를 하라.

In [ ]:
# 충분한 개수의 적절한 테스트를 하라.

In [ ]:
# 충분한 개수의 적절한 테스트를 하라.

## 7. (10점) 위 `lcsstrSS`에 메모해두기 적용
`lcsstrSS` 함수에 메모해두기를 적용한 `lcsstrSSM` 함수를 작성하라.

당연히 `functools`의 `@cache` 매크로를 활용하면 안되고,
수업시간에 했던 것처럼 메모해두는 용도의 딕셔너리와
내부 함수 `rec`을 정의하여 `lcsstrSSM`의 구조를 되도록
따르면서 작성하라.  

함수를 작성한 다음 충분한 개수의 적절한 테스트를 하라.



In [ ]:
def lcsstrSSM(s1:str, s2:str) -> set[str]:
    # 메모해두기에 활용할 딕셔너리 여기에 초기화
    def rec(s1:str, s2:str) -> int:
        pass # lcsstr의 구조를 따라 작성
    # rec 함수 호출
    return rec(s1, s2)

In [ ]:
# 충분한 개수의 적절한 테스트를 하라.

In [ ]:
# 충분한 개수의 적절한 테스트를 하라.

In [ ]:
# 충분한 개수의 적절한 테스트를 하라.

In [ ]:
# 충분한 개수의 적절한 테스트를 하라.

-----
기억하라 중간고사 마지막 문제를 그리고 업그레이드하라!

In [ ]:
from typing import *

In [ ]:
def merge(ls: list, rs: list) -> list:
    ss = []
    while ls!=[] and rs!=[]:
        if ls[0] <= rs[0]:
            ss.append( ls[0] )  # ss = ss + [ ls[0] ]
            ls = ls[1:]
        else: # left[0] > right[0]
            ss.append( rs[0] )  # ss = ss + [ rs[0] ]
            rs = rs[1:]
    # left==[] or right==[]
    return ss + ls + rs

In [ ]:
def mergeAdjacent(xss: list[list]) -> None:
    for k in range(len(xss)//2 - 1, -1, -1):
        xss[2*k:2*k+2] = [ merge(xss[2*k],xss[2*k+1]) ]

In [ ]:
def msort(xs: list) -> list:
    if len(xs) > 1:
        xss = [ [x] for x in xs ] # 원소 한개로 이루어진 리스트들의 리스트
        while len(xss) > 1:
            mergeAdjacent(xss)
        return xss[0]
    else: # 길이가 1이하이면 그대로 이미 정렬된 상태나 마찬가지
        return xs

In [ ]:
msort([])

[]

In [ ]:
msort([1])

[1]

In [ ]:
msort([32,23,18,7,11,99,55])

[7, 11, 18, 23, 32, 55, 99]

## 8. (10점) 위의 상향식 merge sort 구현에 `key` 파라메터를 추가
표준라이브러리 `sorted` 함수처럼

In [ ]:
# def my_merge( ... )
def my_merge(ls: list, rs: list, key) -> list:
    ss = []
    while ls!=[] and rs!=[]:
        if key(ls[0]) <= key(rs[0]):
            ss.append( ls[0] )  # ss = ss + [ ls[0] ]
            ls = ls[1:]
        else: # left[0] > right[0]
            ss.append( rs[0] )  # ss = ss + [ rs[0] ]
            rs = rs[1:]
    # left==[] or right==[]
    return ss + ls + rs
# def my_mergeAdjacent( ... )
def my_mergeAdjacent(xss: list[list], key) -> None:
    for k in range(len(xss)//2 - 1, -1, -1):
        xss[2*k:2*k+2] = [ my_merge(xss[2*k],xss[2*k+1], key) ]

# def my_msort( xs, key=(lambda x: x) )
def my_msort(xs: list, key=(lambda x: x) ) -> list:
    if len(xs) > 1:
        xss = [ [x] for x in xs ] # 원소 한개로 이루어진 리스트들의 리스트
        while len(xss) > 1:
            my_mergeAdjacent(xss, key)
        return xss[0]
    else: # 길이가 1이하이면 그대로 이미 정렬된 상태나 마찬가지
        return xs

In [ ]:
my_msort([32,23,18,7,11,99,55], lambda x : -x)

[99, 55, 32, 23, 18, 11, 7]

In [ ]:
my_msort([20,18,6,53,201,815,60], lambda x : -x)

[815, 201, 60, 53, 20, 18, 6]

In [ ]:
my_msort([32,23,18,7,11,99,55], lambda x : x)

[7, 11, 18, 23, 32, 55, 99]

## 9. (5점) 문자열 길이를 기준으로 정렬하되 길이가 같은 문자열끼리는 사전 역순으로
그런 식으로 정렬되도록 위에서 작성한 my_msort를 활용해 정렬하는 실행 테스트를 작성하라

In [ ]:
my_msort(['this', 'is', 'very', 'hard', 'to', 'change', 'that'], lambda x: (len(x), -ord(x[0])))

['to', 'is', 'very', 'this', 'that', 'hard', 'change']